In [ ]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import ttest_1samp, shapiro
from sklearn.feature_selection import chi2

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import recall_score, make_scorer, roc_auc_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from graphviz import Source
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG, display
# import shap

import warnings
warnings.filterwarnings('ignore')
from scipy.stats.mstats import winsorize
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from sklearn.decomposition import PCA
# from pycaret.classification import *
import lightgbm as lgb
from sklearn.metrics import recall_score
import six
import sys
sys.modules['sklearn.externals.six'] = six
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize, space
from functools import partial

In [ ]:
# df = pd.read_csv('/content/sample_data/Best_dataset_fp.csv')
# df.drop('Unnamed: 0', axis=1, inplace=True)
df = pd.read_csv('/content/sample_data/Best_dataset2.csv')

In [ ]:
# Split the DataFrame into training set (90%) and test set (10%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
X = train_df.drop(['TenYearCHD_1'], axis = 1)
y = train_df['TenYearCHD_1']

### LighGBM

In [ ]:
X = train_df.drop(['TenYearCHD_1'], axis = 1)
y = train_df['TenYearCHD_1']
X = X.reset_index(drop=True)

def optimize(params, X, y):
  params = dict(zip(param_names, params))
  model = lgb.LGBMClassifier(**params)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
  scores = []
  for train_idx, val_idx in cv.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    model.fit(X_train, y_train, early_stopping_rounds=10,
              eval_set=[(X_val, y_val)], verbose=False)
    y_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, y_pred)
    scores.append(score)
  return -1.0 * np.mean(scores)
# Define the parameter space and names
param_space = [
    space.Integer(3, 15, name="max_depth"),
    space.Integer(100, 600, name="n_estimators"),
    space.Real(0.01, 1, prior="uniform", name="learning_rate"),
    space.Real(0.1, 1, prior="uniform", name="subsample"),
    space.Real(0.1, 1, prior="uniform", name="colsample_bytree"),
    space.Real(0.1, 10, prior="uniform", name="min_child_weight")
    ]
param_names = ["max_depth", "n_estimators", "learning_rate", "subsample", "colsample_bytree", "min_child_weight"]

# Define the optimization function with partial arguments
optimization_function = partial(optimize, X=X, y=y)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=200,
    n_random_starts=200,
    verbose=10,
    acq_func='EI'
)
# Print the best hyperparameters found
print(dict(zip(param_names, result.x)))
print(f'Best ROC-AUC Score: {-1.0 * result.fun}')

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 5.0826
Function value obtained: -0.6771
Current minimum: -0.6771
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.4856
Function value obtained: -0.6860
Current minimum: -0.6860
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.3294
Function value obtained: -0.6350
Current minimum: -0.6860
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.3133
Function value obtained: -0.6924
Current minimum: -0.6924
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.2762
Function value obtained: -0.6835
Current minimum: -0.6924
Iteration No: 6 started. 

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'max_depth': 3,
          'n_estimators': 173,
          'learning_rate': 0.26658391864937014,
          'subsample': 0.27417395433428393,
          'colsample_bytree': 0.37678704205444047,
          'min_child_weight': 8.827872937189104}


# Train the model
model = lgb.LGBMClassifier(**params)
model.fit(X, y)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.6730429464528404


Optuna TPE

In [ ]:
!pip install optuna

In [ ]:
import optuna
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from functools import partial

X = train_df.drop(['TenYearCHD'], axis=1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)

def optimize(trial, X, y):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10)
    }
    model = lgb.LGBMClassifier(**params)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
    scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)], verbose=False)
        y_pred = model.predict_proba(X_val)[:, 1]
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
    return -1.0 * np.mean(scores)

# Create the objective function with partial arguments
objective = partial(optimize, X=X, y=y)

# Create the study object with the TPE sampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())

# Optimize the objective function
study.optimize(objective, n_trials=200, n_jobs=-1)

# Print the best hyperparameters found
best_params = study.best_params
best_score = study.best_value
print("Best Parameters:", best_params)
print("Best ROC-AUC Score:", best_score)


[I 2023-05-25 13:59:57,570] A new study created in memory with name: no-name-c48fd018-681c-416b-abb7-3fe0a251acf4
[I 2023-05-25 13:59:59,531] Trial 1 finished with value: -0.6601684318970503 and parameters: {'max_depth': 8, 'n_estimators': 174, 'learning_rate': 0.7878747640239373, 'subsample': 0.2592273280165252, 'colsample_bytree': 0.8806143626087539, 'min_child_weight': 2.2382901801725468}. Best is trial 1 with value: -0.6601684318970503.
[I 2023-05-25 13:59:59,961] Trial 0 finished with value: -0.6653024990360518 and parameters: {'max_depth': 9, 'n_estimators': 470, 'learning_rate': 0.48493139233540294, 'subsample': 0.8314171009100851, 'colsample_bytree': 0.2605551341091043, 'min_child_weight': 8.29175647100937}. Best is trial 1 with value: -0.6601684318970503.
[I 2023-05-25 14:00:01,207] Trial 2 finished with value: -0.606153344097423 and parameters: {'max_depth': 7, 'n_estimators': 476, 'learning_rate': 0.8653694779858707, 'subsample': 0.4874937843909558, 'colsample_bytree': 0.301

Best Parameters: {'max_depth': 9, 'n_estimators': 429, 'learning_rate': 0.785461337067571, 'subsample': 0.7716244833348449, 'colsample_bytree': 0.2970813261191314, 'min_child_weight': 0.27106750258415757}
Best ROC-AUC Score: -0.5888968836353705


In [ ]:



X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'max_depth': 9, 'n_estimators': 429, 
          'learning_rate': 0.785461337067571, 
          'subsample': 0.7716244833348449, 
          'colsample_bytree': 0.2970813261191314, 
          'min_child_weight': 0.27106750258415757}


# Train the model
model = lgb.LGBMClassifier(**params)
model.fit(X, y)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.5996364501223158


Optuna CMA es

In [ ]:
import optuna
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from functools import partial

X = train_df.drop(['TenYearCHD'], axis=1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)

def optimize(trial, X, y):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'n_estimators': trial.suggest_int('n_estimators', 100, 600),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10)
    }
    model = lgb.LGBMClassifier(**params)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
    scores = []
    for train_idx, val_idx in cv.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        model.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_val, y_val)], verbose=False)
        y_pred = model.predict_proba(X_val)[:, 1]
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
    return -1.0 * np.mean(scores)

# Create the objective function with partial arguments
objective = partial(optimize, X=X, y=y)

# Create the study object with the CMA-ES sampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.CmaEsSampler())

# Optimize the objective function
study.optimize(objective, n_trials=200, n_jobs=-1)

# Print the best hyperparameters found
best_params = study.best_params
best_score = study.best_value
print("Best Parameters:", best_params)
print("Best ROC-AUC Score:", best_score)


[I 2023-05-25 14:05:27,840] A new study created in memory with name: no-name-41dd3321-b802-4f5b-ad03-1dd1e2f5bee8
[I 2023-05-25 14:05:29,311] Trial 0 finished with value: -0.7060940331598228 and parameters: {'max_depth': 3, 'n_estimators': 507, 'learning_rate': 0.49476589130576076, 'subsample': 0.9020776914051981, 'colsample_bytree': 0.6597056269275677, 'min_child_weight': 5.045128028844567}. Best is trial 0 with value: -0.7060940331598228.
[I 2023-05-25 14:05:29,893] Trial 1 finished with value: -0.6802005775657091 and parameters: {'max_depth': 8, 'n_estimators': 477, 'learning_rate': 0.457262131086608, 'subsample': 0.2195707793567391, 'colsample_bytree': 0.7529799156865982, 'min_child_weight': 8.414283199551413}. Best is trial 1 with value: -0.6802005775657091.
[I 2023-05-25 14:05:31,020] Trial 2 finished with value: -0.6834266354990038 and parameters: {'max_depth': 9, 'n_estimators': 403, 'learning_rate': 0.43185778642083034, 'subsample': 0.5926737137629609, 'colsample_bytree': 0.58

Best Parameters: {'max_depth': 9, 'n_estimators': 393, 'learning_rate': 0.7836959863238611, 'subsample': 0.223447751878518, 'colsample_bytree': 0.32143814302439705, 'min_child_weight': 1.0809143939825714}
Best ROC-AUC Score: -0.5889490091414433


In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'max_depth': 9, 'n_estimators': 393, 
          'learning_rate': 0.7836959863238611, 'subsample': 0.223447751878518, 
          'colsample_bytree': 0.32143814302439705, 'min_child_weight': 1.0809143939825714}

# Train the model
model = lgb.LGBMClassifier(**params)
model.fit(X, y)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.6265459363957597


### XGBOOST

In [ ]:
X = train_df.drop(['TenYearCHD'], axis = 1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)

def optimize(params, X, y):
  params = dict(zip(param_names, params))
  model = xgb.XGBClassifier(**params)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
  scores = []
  for train_idx, val_idx in cv.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    model.fit(X_train, y_train, early_stopping_rounds=10,
              eval_set=[(X_val, y_val)], verbose=False)
    y_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, y_pred)
    scores.append(score)
  return -1.0 * np.mean(scores)
# Define the parameter space and names

param_space = [
    space.Integer(3, 15, name="max_depth"),
    space.Integer(100, 1000, name="n_estimators"),
    space.Real(0.01, 1, prior="uniform", name="learning_rate"),
    space.Real(0.1, 1, prior="uniform", name="subsample"),
    space.Real(0.1, 1, prior="uniform", name="colsample_bytree"),
    space.Real(0.1, 10, prior="uniform", name="min_child_weight")
    ]
param_names = ["max_depth", "n_estimators", "learning_rate", "subsample", "colsample_bytree", "min_child_weight"]

# Define the optimization function with partial arguments
optimization_function = partial(optimize, X=X, y=y)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=200,
    n_random_starts=200,
    verbose=10,
    acq_func='EI'
)
# Print the best hyperparameters found
print(dict(zip(param_names, result.x)))
print(f'Best ROC-AUC Score: {-1.0 * result.fun}')

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 6.1596
Function value obtained: -0.6906
Current minimum: -0.6906
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.1131
Function value obtained: -0.6755
Current minimum: -0.6906
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 1.5695
Function value obtained: -0.6952
Current minimum: -0.6952
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.7505
Function value obtained: -0.6847
Current minimum: -0.6952
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.5306
Function value obtained: -0.6526
Current minimum: -0.6952
Iteration No: 6 started. 

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'max_depth': 4, 'n_estimators': 507, 'learning_rate': 0.08488585554676588, 
          'subsample': 0.7428342774870389, 'colsample_bytree': 0.8704321827011343, 'min_child_weight': 7.467421949775829}



# Train the model
model = xgb.XGBClassifier(**params)
model.fit(X, y)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.6808405816798042


### GB

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
X = train_df.drop(['TenYearCHD'], axis = 1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)
def optimize(params, X, y):
  params = dict(zip(param_names, params))
  model = GradientBoostingClassifier(**params, n_iter_no_change=10)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
  scores = []
  for train_idx, val_idx in cv.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, y_pred)
    scores.append(score)
  return -1.0 * np.mean(scores)
# Define the parameter space and names

param_space = [
    Real(0.01, 1, prior='log-uniform', name='learning_rate'),
    Integer(2, 10, name='max_depth'),
    Real(0.1, 1, prior='uniform', name='subsample'),
    Real(0.1, 1, prior='uniform', name='max_features'),
    Integer(2, 100, name='n_estimators'),
    Integer(2, 50, name='min_samples_split'),
    Integer(1, 20, name='min_samples_leaf')
]

param_names = ["learning_rate", "max_depth", "subsample", "max_features", "n_estimators", "min_samples_split", "min_samples_leaf"]

# Define the optimization function with partial arguments
optimization_function = partial(optimize, X=X, y=y)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=50,
    n_random_starts=50,
    verbose=10,
    acq_func='EI'
)
# Print the best hyperparameters found
print(dict(zip(param_names, result.x)))
print(f'Best ROC-AUC Score: {-1.0 * result.fun}')

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.8573
Function value obtained: -0.7024
Current minimum: -0.7024
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.6492
Function value obtained: -0.7047
Current minimum: -0.7047
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.2132
Function value obtained: -0.7029
Current minimum: -0.7047
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 1.0672
Function value obtained: -0.6888
Current minimum: -0.7047
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 1.5988
Function value obtained: -0.7025
Current minimum: -0.7047
Iteration No: 6 started. 

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'learning_rate': 0.026788911880272295, 'max_depth': 3, 
          'subsample': 0.21130099384109907, 'max_features': 0.6159058226808146, 'n_estimators': 56, 'min_samples_split': 11, 'min_samples_leaf': 8}

# Train the model
model = GradientBoostingClassifier(**params)
model.fit(X, y)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.6999439385702636


### Logistic Regression

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
X = train_df.drop(['TenYearCHD'], axis = 1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)
def optimize(params, X, y):
  params = dict(zip(param_names, params))
  model = LogisticRegression(**params)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
  scores = []
  for train_idx, val_idx in cv.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, y_pred)
    scores.append(score)
  return -1.0 * np.mean(scores)
# Define the parameter space and names

param_space = [
    Categorical(categories=[0.01, 0.1, 1, 10, 100], name='C'),
    Categorical(categories=['l1', 'l2'], name='penalty'),
    Categorical(categories=['liblinear', 'saga'], name='solver')
    ]

param_names = ["C", "penalty", "solver"]

# Define the optimization function with partial arguments
optimization_function = partial(optimize, X=X, y=y)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=50,
    n_random_starts=50,
    verbose=10,
    acq_func='EI'
)
# Print the best hyperparameters found
print(dict(zip(param_names, result.x)))
print(f'Best ROC-AUC Score: {-1.0 * result.fun}')

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.2516
Function value obtained: -0.7211
Current minimum: -0.7211
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.2546
Function value obtained: -0.7208
Current minimum: -0.7211
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.2436
Function value obtained: -0.7222
Current minimum: -0.7222
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.2586
Function value obtained: -0.7208
Current minimum: -0.7222
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.1636
Function value obtained: -0.7210
Current minimum: -0.7222
Iteration No: 6 started. 

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'C': 1.0, 'penalty': 'l2', 'solver': 'saga'}

# Train the model
model = LogisticRegression(**params)
model.fit(X, y)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.7011076379450938


### KNN

In [ ]:
X = train_df.drop(['TenYearCHD'], axis = 1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)

def optimize(params, X, y):
  n_neighbors = int(params[0])
  weights = params[1]
  # metric = str(params[2])
  model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
  scores = []
  for train_idx, val_idx in cv.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, y_pred)
    scores.append(score)
  return -1.0 * np.mean(scores)

# Define the parameter space and names

param_space = [
    Categorical(categories=[1, 3, 5, 7, 9, 11, 13, 15, 17, 19], name='n_neighbors'),
    Categorical(categories=['uniform', 'distance'], name='weights')
    # Categorical(categories=['euclidean', 'manhattan', 'minkowski'], name='metric')
]

param_names = ["n_neighbors", "weights", "metric"]

# Define the optimization function with partial arguments
optimization_function = partial(optimize, X=X, y=y)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=100,
    n_random_starts=100,
    verbose=10,
    acq_func='EI'
)
# Print the best hyperparameters found
print(dict(zip(param_names, result.x)))
print(f'Best ROC-AUC Score: {-1.0 * result.fun}')

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 1.1404
Function value obtained: -0.6630
Current minimum: -0.6630
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.1256
Function value obtained: -0.6506
Current minimum: -0.6630
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.7072
Function value obtained: -0.5991
Current minimum: -0.6630
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.4472
Function value obtained: -0.5959
Current minimum: -0.6630
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.5055
Function value obtained: -0.6145
Current minimum: -0.6630
Iteration No: 6 started. 

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'n_neighbors': 19, 'weights': 'distance'}

# Train the model
model = KNeighborsClassifier(**params)
model.fit(X, y)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.6672839086708344


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X = train_df.drop(['TenYearCHD'], axis = 1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)


def optimize(params, X, y):
  params = dict(zip(param_names, params))
  model = RandomForestClassifier(**params)
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
  scores = []
  for train_idx, val_idx in cv.split(X, y):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    score = roc_auc_score(y_val, y_pred)
    scores.append(score)
  return -1.0 * np.mean(scores)

param_space = [
    Integer(10, 100, name='n_estimators'),
    Integer(2, 20, name='max_depth'),
    Categorical(categories=['gini', 'entropy'], name='criterion')
]

param_names = ["n_estimators", "max_depth", "criterion"]

optimization_function = partial(optimize, X=X, y=y)

result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=50,
    n_random_starts=50,
    verbose=10,
    acq_func='EI'
)

print(dict(zip(param_names, result.x)))
print(f'Best ROC-AUC Score: {-1.0 * result.fun}')

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 6.9058
Function value obtained: -0.7085
Current minimum: -0.7085
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 3.5121
Function value obtained: -0.6866
Current minimum: -0.7085
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.4531
Function value obtained: -0.7068
Current minimum: -0.7085
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 7.8704
Function value obtained: -0.6922
Current minimum: -0.7085
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 4.4191
Function value obtained: -0.6895
Current minimum: -0.7085
Iteration No: 6 started. 

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

# Assuming you have your training and evaluation data as X_train, y_train, X_eval, y_eval, and test data as X_test, y_test

# Define the parameters
params = {'n_estimators': 20, 'max_depth': 7, 'criterion': 'gini'}

# Train the model
model = RandomForestClassifier(**params)
model.fit(X, y)

# Make predictions
y_pred = model.predict_proba(X_test)[:, 1]

# Calculate the AUC
auc = roc_auc_score(y_test, y_pred)
print("AUC:", auc)

AUC: 0.6872961402555042


### Neural Network

In [ ]:
X = train_df.drop(['TenYearCHD'], axis = 1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

import numpy as np
from functools import partial
from skopt import gp_minimize
from skopt import space
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Define the neural network model
def create_model(hidden_units, learning_rate):
    model = Sequential()
    model.add(Dense(units=hidden_units, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(units=1, activation='sigmoid'))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    return model

# Define the optimization function
def optimize_nn(params):
    hidden_units = params[0]
    learning_rate = params[1]
    
    model = create_model(hidden_units=hidden_units, learning_rate=learning_rate)
    
    scores = []
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)
    
    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        
        y_pred = model.predict(X_val)[:, 0]
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
    
    return -np.mean(scores)

# Define the parameter space and names
param_space = [
    space.Integer(10, 300, name='hidden_units'),
    space.Real(0.0001, 0.1, prior='log-uniform', name='learning_rate')
]
param_names = ['hidden_units', 'learning_rate']

# Define the optimization function with partial arguments
optimization_function = partial(optimize_nn)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=30,
    n_random_starts=30,
    verbose=1,
    acq_func='EI',
    random_state=42
)

# Print the best hyperparameters found
best_params = dict(zip(param_names, result.x))
print(best_params)
print(f'Best ROC-AUC Score: {-result.fun}')


Iteration No: 1 started. Evaluating function at random point.
17/17 [==============================] - 0s 3ms/step
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 32.3382
Function value obtained: -0.7247
Current minimum: -0.7247
Iteration No: 2 started. Evaluating function at random point.
17/17 [==============================] - 0s 2ms/step
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 24.9058
Function value obtained: -0.7472
Current minimum: -0.7472
Iteration No: 3 started. Evaluating function at random point.
17/17 [==============================] - 0s 1ms/step
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 26.8740
Function value obtained: -0.6969
Current minimum: -0.7472
Iteration No: 4 started. Evaluating function at random point.
17/17 [==============================] - 0s 2ms/step
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 25.1126
Function value obtained: -0.7364
Current minimum: -0.7472
Iter

In [ ]:
X_test = test_df.drop(['TenYearCHD'], axis = 1)
y_test = test_df['TenYearCHD']
from sklearn.metrics import roc_auc_score

optimized_model = Sequential()
optimized_model.add(Dense(units=236, activation='relu', input_dim=X.shape[1]))
optimized_model.add(Dense(units=1, activation='sigmoid'))
optimizer = Adam(learning_rate=0.006173770394704579)
optimized_model.compile(optimizer=optimizer, loss='binary_crossentropy')


# Train the model on the training data
optimized_model.fit(X, y, epochs=10, batch_size=32, verbose=1)

# Make predictions on X_test
y_test_pred_probs = optimized_model.predict(X_test)[:, 0]

# Calculate ROC-AUC score
roc_auc = roc_auc_score(y_test, y_test_pred_probs)

print("ROC-AUC Score:", roc_auc)

Epoch 1/10
84/84 [==============================] - 1s 3ms/step - loss: 0.4385
Epoch 2/10
84/84 [==============================] - 0s 3ms/step - loss: 0.3969
Epoch 3/10
84/84 [==============================] - 0s 3ms/step - loss: 0.3881
Epoch 4/10
84/84 [==============================] - 0s 4ms/step - loss: 0.3852
Epoch 5/10
84/84 [==============================] - 0s 3ms/step - loss: 0.3842
Epoch 6/10
84/84 [==============================] - 0s 3ms/step - loss: 0.3841
Epoch 7/10
84/84 [==============================] - 0s 4ms/step - loss: 0.3808
Epoch 8/10
84/84 [==============================] - 0s 2ms/step - loss: 0.3855
Epoch 9/10
84/84 [==============================] - 0s 3ms/step - loss: 0.3813
Epoch 10/10
21/21 [==============================] - 0s 3ms/step
ROC-AUC Score: 0.7033840717586302


### Soft Voting

In [ ]:
import numpy as np
from functools import partial
from skopt import gp_minimize
from skopt import space
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

# Define the wrapper class for Keras models
class KerasWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model
    
    def fit(self, X, y):
        self.model.fit(X, y)
    
    def predict_proba(self, X):
        predictions = self.model.predict(X)
        proba = predictions.flatten()
        proba = np.vstack([1 - proba, proba]).T
        return proba

# Define the base models
neural_network = Sequential()
neural_network.add(Dense(units=236, activation='relu', input_dim=X_train.shape[1]))
neural_network.add(Dense(units=1, activation='sigmoid'))
optimizer = Adam(learning_rate=0.006173770394704579)
neural_network.compile(optimizer=optimizer, loss='binary_crossentropy')

neural_network_model = KerasWrapper(neural_network)

xgboost_model = XGBClassifier(max_depth=4, n_estimators=507,
                              learning_rate=0.08488585554676588, subsample=0.7428342774870389,
                              colsample_bytree=0.8704321827011343, min_child_weight=7.467421949775829)

lightgbm_model = LGBMClassifier(max_depth=3, n_estimators=173,
                                learning_rate=0.26658391864937014,
                                subsample=0.27417395433428393,
                                colsample_bytree=0.37678704205444047,
                                min_child_weight=8.827872937189104)

random_forest_model = RandomForestClassifier(n_estimators=20, max_depth=7, criterion='gini')
lr_model = LogisticRegression(C = 1.0, penalty = 'l2', solver = 'saga')
##############################################################################################################
from mlxtend.classifier import EnsembleVoteClassifier
import numpy as np
# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the voting classifier
voting_classifier = EnsembleVoteClassifier(
    clfs=[
        neural_network_model,
        xgboost_model,
        lightgbm_model,
        random_forest_model,
        lr_model
    ],
    voting='soft',
    verbose=1
)

# Define the optimization function
def optimize_weights(weights, X, y):
    voting_classifier.weights = weights

    scores = []
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

    for train_idx, val_idx in cv.split(X, y):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        voting_classifier.fit(X_train, y_train)
        y_pred = voting_classifier.predict_proba(X_val)[:, 1]  # Use predicted probabilities for AUC
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)

    return -np.mean(scores)

# Example usage
num_classifiers = 5
weights = np.full(num_classifiers, 0.5)  # Initial weights

# Define the parameter space for weights
param_space = [space.Real(0, 1, name='weight_{}'.format(i)) for i in range(num_classifiers)]

# Define the optimization function with partial arguments
optimization_function = partial(optimize_weights, X=X_train, y=y_train)

# Run the optimization using gp_minimize
result = gp_minimize(
    optimization_function,
    dimensions=param_space,
    n_calls=20,
    n_random_starts=20,
    verbose=True
)

# Get the optimal weights
optimal_weights = result.x

print("Optimal weights:", optimal_weights)

Iteration No: 1 started. Evaluating function at random point.
Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 1s 3ms/step - loss: 0.4501
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 6ms/step


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 0s 2ms/step - loss: 0.4338
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 2ms/step


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 0s 2ms/step - loss: 0.4534
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 2ms/step


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 0s 2ms/step - loss: 0.4429
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 2ms/step


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 0s 2ms/step - loss: 0.4430
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 1ms/step


...

Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 0s 2ms/step - loss: 0.4556
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 2ms/step


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 0s 2ms/step - loss: 0.4455
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 2ms/step


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 0s 2ms/step - loss: 0.4505
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 2ms/step


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 0s 2ms/step - loss: 0.4624
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 3ms/step


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
54/54 [==============================] - 1s 3ms/step - loss: 0.4555
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
14/14 [==============================] - 0s 2ms/step
Iteration No: 20 ended. Evaluation done at random point.
Time taken: 31.7983
Function value obtained: -0.6731
Current minimum: -0.7024
Optimal weights: [0.7031865096223286, 0.45725464901581847, 0.014723448242867202, 0.27636134692557307, 0.8056712170488267]


In [ ]:

# Set the optimal weights in the voting classifier
voting_classifier.weights = optimal_weights

# Evaluate the performance on the test set
y_test_pred_probs = voting_classifier.predict_proba(X_test)[:, 1]
test_score = roc_auc_score(y_test, y_test_pred_probs)

print("Optimized weights:", optimal_weights)

print("Test ROC-AUC score:", test_score)


21/21 [==============================] - 0s 2ms/step
Optimized weights: [0.7031865096223286, 0.45725464901581847, 0.014723448242867202, 0.27636134692557307, 0.8056712170488267]
Test ROC-AUC score: 0.6808915466159282


In [ ]:
np.sum([0.7031865096223286, 0.45725464901581847, 0.014723448242867202, 0.27636134692557307, 0.8056712170488267])

2.257197170855414

In [ ]:
optimal_weights

[0.7031865096223286,
 0.45725464901581847,
 0.014723448242867202,
 0.27636134692557307,
 0.8056712170488267]

In [ ]:

# Set the optimal weights in the voting classifier
voting_classifier.weights = [0.9031865096223286, 0.45725464901581847, 0.014723448242867202, 0.27636134692557307, 0.8056712170488267]

# Evaluate the performance on the test set
y_test_pred_probs = voting_classifier.predict_proba(X_test)[:, 1]
test_score = roc_auc_score(y_test, y_test_pred_probs)

print("Optimized weights:", optimal_weights)

print("Test ROC-AUC score:", test_score)


21/21 [==============================] - 0s 4ms/step
Optimized weights: [0.7031865096223286, 0.45725464901581847, 0.014723448242867202, 0.27636134692557307, 0.8056712170488267]
Test ROC-AUC score: 0.6757440880674096


### Stacking

In [ ]:
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from mlxtend.classifier import StackingCVClassifier

X = train_df.drop(['TenYearCHD'], axis = 1)
y = train_df['TenYearCHD']
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

# Define the wrapper class for Keras models
class KerasWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model
    
    def fit(self, X, y):
        self.model.fit(X, y)
    
    def predict_proba(self, X):
        predictions = self.model.predict(X)
        proba = predictions.flatten()
        proba = np.vstack([1 - proba, proba]).T
        return proba

# Define the base models
neural_network = Sequential()
neural_network.add(Dense(units=236, activation='relu', input_dim=X_train.shape[1]))
neural_network.add(Dense(units=1, activation='sigmoid'))
optimizer = Adam(learning_rate=0.006173770394704579)
neural_network.compile(optimizer=optimizer, loss='binary_crossentropy')

neural_network_model = KerasWrapper(neural_network)

xgboost_model = XGBClassifier(max_depth=4, n_estimators=507,
                              learning_rate=0.08488585554676588, subsample=0.7428342774870389,
                              colsample_bytree=0.8704321827011343, min_child_weight=7.467421949775829)

lightgbm_model = LGBMClassifier(max_depth=3, n_estimators=173,
                                learning_rate=0.26658391864937014,
                                subsample=0.27417395433428393,
                                colsample_bytree=0.37678704205444047,
                                min_child_weight=8.827872937189104)

random_forest_model = RandomForestClassifier(n_estimators=20, max_depth=7, criterion='gini')
lr_model = LogisticRegression(C = 1.0, penalty = 'l2', solver = 'saga')

# Define meta model
meta_model = XGBClassifier(max_depth= 3, n_estimators= 100, learning_rate= 0.06, subsample= 0.7, colsample_bytree= 0.4, reg_alpha=0.3, reg_lambda=0.3,min_child_weight= 10)

# Define stratified k-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define stacking classifier with base models and meta model
stacking_clf_xgb4 = StackingCVClassifier(classifiers=[neural_network_model, xgboost_model, lightgbm_model, random_forest_model, lr_model],
                                    meta_classifier=meta_model,
                                    cv=5,
                                    stratify=True,
                                    shuffle=True,
                                    use_probas=True,
                                    use_features_in_secondary=True,
                                    verbose=2)

# Define metric as ROC
metric = roc_auc_score

# Fit the stacking classifier
stacking_clf_xgb4.fit(X.values, y.values)

# Calculate the score
score = metric(y, stacking_clf_xgb4.predict_proba(X)[:, 1])


# Print the score
print(f"ROC AUC score: {score}")



Fitting 5 classifiers...
Fitting classifier1: keraswrapper (1/5)
KerasWrapper(model=<keras.engine.sequential.Sequential object at 0x7fe9f4c48f10>)
Training and fitting fold 1 of 5...
17/17 [==============================] - 0s 2ms/step
Training and fitting fold 2 of 5...
17/17 [==============================] - 0s 1ms/step
Training and fitting fold 3 of 5...
17/17 [==============================] - 0s 1ms/step
Training and fitting fold 4 of 5...
17/17 [==============================] - 0s 2ms/step
Training and fitting fold 5 of 5...
17/17 [==============================] - 0s 1ms/step
Fitting classifier2: xgbclassifier (2/5)
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8704321827011343, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
         

In [ ]:

# Evaluate the performance on the test set
y_test_pred_probs = stacking_clf_xgb4.predict_proba(X_test)[:, 1]
test_score = roc_auc_score(y_test, y_test_pred_probs)


print("Test ROC-AUC score:", test_score)


21/21 [==============================] - 0s 2ms/step
Test ROC-AUC score: 0.7047431367219353


### Old Soft Voting

In [ ]:
from mlxtend.classifier import EnsembleVoteClassifier


# Define the wrapper class for Keras models
class KerasWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model):
        self.model = model
    
    def fit(self, X, y):
        self.model.fit(X, y)
    
    def predict_proba(self, X):
        predictions = self.model.predict(X)
        proba = predictions.flatten()
        proba = np.vstack([1 - proba, proba]).T
        return proba

# Define the base models
neural_network = Sequential()
neural_network.add(Dense(units=236, activation='relu', input_dim=X_train.shape[1]))
neural_network.add(Dense(units=1, activation='sigmoid'))
optimizer = Adam(learning_rate=0.006173770394704579)
neural_network.compile(optimizer=optimizer, loss='binary_crossentropy')

neural_network_model = KerasWrapper(neural_network)

xgboost_model = XGBClassifier(max_depth=4, n_estimators=507,
                              learning_rate=0.08488585554676588, subsample=0.7428342774870389,
                              colsample_bytree=0.8704321827011343, min_child_weight=7.467421949775829)

lightgbm_model = LGBMClassifier(max_depth=3, n_estimators=173,
                                learning_rate=0.26658391864937014,
                                subsample=0.27417395433428393,
                                colsample_bytree=0.37678704205444047,
                                min_child_weight=8.827872937189104)

random_forest_model = RandomForestClassifier(n_estimators=20, max_depth=7, criterion='gini')
lr_model = LogisticRegression(C = 1.0, penalty = 'l2', solver = 'saga')


# # set the weights for each classifier
# weights = [0.99, 0.993, 1.004, 1.006, 1.007]
weights = [1.5, 1, 1, 1, 1.5]
#0.121


classifiers = [neural_network_model, xgboost_model, lightgbm_model, random_forest_model, lr_model]

# create the ensemble classifier
ensemble_SVW_O = EnsembleVoteClassifier(clfs=classifiers, voting='soft', weights = weights, verbose=1)

# fit the ensemble classifier on the training data
ensemble_SVW_O.fit(X, y)


# Calculate the score
score = metric(y, stacking_clf_xgb4.predict_proba(X)[:, 1])


# Print the score
print(f"ROC AUC score: {score}")


Fitting 5 classifiers...
Fitting clf1: keraswrapper (1/5)
84/84 [==============================] - 1s 4ms/step - loss: 0.4364
Fitting clf2: xgbclassifier (2/5)
Fitting clf3: lgbmclassifier (3/5)
Fitting clf4: randomforestclassifier (4/5)
Fitting clf5: logisticregression (5/5)
84/84 [==============================] - 0s 1ms/step
ROC AUC score: 0.782939367311072


In [ ]:

# Evaluate the performance on the test set
y_test_pred_probs = ensemble_SVW_O.predict_proba(X_test)[:, 1]
test_score = roc_auc_score(y_test, y_test_pred_probs)


print("Test ROC-AUC score:", test_score)


21/21 [==============================] - 0s 1ms/step
Test ROC-AUC score: 0.6903880130470237
